# 最新日志处理

In [115]:
import time
from datetime import datetime
import redis
import bisect

In [8]:
def log_recent(conn, name, message, serverity='info',pipe=None):
    destination = 'recent:%s:%s' %(name, serverity)
    message = time.asctime() + ' ' + message
    pipe = pipe or conn.pipeline()
    pipe.lpush(destination, message)
    pipe.ltrim(destination, 0, 99)
    pipe.execute()

In [15]:
conn = redis.Redis(decode_responses=True)
log_recent(conn, 'hit', '这是一条点击日志')

In [16]:
conn.lrange('recent:hit:info',0,-1)

['Tue Nov  5 20:20:32 2019 这是一条点击日志']

In [66]:
def log_common(conn, name, message, serverity='info', timeout=5):
    destination = 'common:%s:%s' %(name, serverity)
    start_key = destination+':start'
    pipe = conn.pipeline()
    end = time.time() + timeout
    while time.time() < end:
        try:
            pipe.watch(start_key)
            now = datetime.now().timetuple()
            hour_start =  datetime(*now[:4]).isoformat()
            
            existing = pipe.get(start_key)
            pipe.multi()
            if existing and existing < hour_start:
                pipe.rename(destination, destination+':last')
                pipe.rename(start_key, start_key+':pstart')
                pipe.set(start_key, hour_start)
            elif not existing:
                pipe.set(start_key, hour_start)
            pipe.zincrby(destination, 1,message)
            log_recent(pipe, name, message, serverity, pipe)
            return
        except redis.exceptions.WatchError:
            continue

In [67]:
log_common(conn, 'click', '这是一个点击时间')

In [68]:
conn.zrange('common:click:info',0,-1,withscores=True)

[('这是一个点击时间', 3.0)]

In [69]:
conn.lrange('recent:click:info',0,-1)

['Tue Nov  5 20:50:57 2019 这是一个点击时间',
 'Tue Nov  5 20:47:02 2019 这是一个点击时间',
 'Tue Nov  5 20:45:34 2019 这是一个点击时间',
 'Tue Nov  5 20:45:09 2019 这是一个点击时间']

In [70]:
now = datetime.now().timetuple()
hour_start =  datetime(*now[:4]).isoformat()

In [71]:
hour_start

'2019-11-05T20:00:00'

In [72]:
conn.get('common:click:info:start')

'2019-11-05T20:00:00'

# 计数日志

In [76]:
PRECISION = [1, 5, 60, 300, 3600, 18000, 86400]

def update_counter(conn, name, count=1, now=None):
    now = now or time.time()
    pipe = conn.pipeline()
    for prec in PRECISION:
        pnow = int(now/prec) * prec
        hash = '%s:%s' %(prec, name)
        pipe.zadd('know:', {hash:0})
        pipe.hincrby('count:'+hash, pnow, count)
    pipe.execute()

In [100]:
update_counter(conn, 'hit')

In [101]:
conn.zrange('know:',0,-1,withscores=True)

[('18000:hit', 0.0),
 ('1:hit', 0.0),
 ('300:hit', 0.0),
 ('3600:hit', 0.0),
 ('5:hit', 0.0),
 ('60:hit', 0.0),
 ('86400:hit', 0.0)]

In [128]:
conn.hgetall('count:1:hit')

{'1572959063': '1',
 '1572959175': '1',
 '1572959199': '1',
 '1572959221': '1',
 '1572959227': '1',
 '1572959232': '1'}

In [103]:
conn.hgetall('count:60:hit')

{'1572959040': '1', '1572959160': '2', '1572959220': '3'}

In [113]:
def get_counter(conn, name, precision):
    hash = '%s:%s' %(precision, name)
    data = conn.hgetall('count:' + hash)
    to_return = []
    for key, value in data.items():
        to_return.append((int(key), int(value)))
    to_return.sort()
    return to_return

In [114]:
get_counter(conn, 'hit','5')

[(1572959060, 1),
 (1572959175, 1),
 (1572959195, 1),
 (1572959220, 1),
 (1572959225, 1),
 (1572959230, 1)]

In [125]:
QUIT = False
SAMPLE_COUNT = 100
def clean_counters(conn):
    pipe = conn.pipeline(True)
    passes = 0
    while not QUIT:
        start = time.time()
        index = 0
        while index < conn.zcard('know:'):
            hash = conn.zrange('know:', index, index)
            index += 1
            if not hash:
                break
            hash = hash[0]
            prec = int(hash.partition(':')[0])
            bprec = int(prec // 60) or 1
            if passes % bprec:
                continue
            hkey = 'count:'+hash
            cutoff = time.time() - SAMPLE_COUNT * prec
            samples = list(map(int, conn.hkeys(hkey)))
            samples.sort()
            remove = bisect.bisect_right(samples, cutoff)
            if remove:
                conn.hdel(hkey, * samples[:remove])
                if remove == len(remove):
                    try:
                        pipe.watch(hkey)
                        if not pipe.hlen(hkey):
                            pipe.multi()
                            pipe.zrem('know:',hash)
                            pipe.execute()
                            index -=1
                        else:
                            pipe.unwatch()
                    except redis.exceptions.WatchError:
                        pass
            passes += 1
            duration = min(int(time.time()-start)+1, 60)
            time.sleep(max(60-duration,1))

In [129]:
import _thread

In [130]:
_thread.start_new_thread(clean_counters, (conn,))

123145409945600

In [144]:
conn.hgetall('count:5:hit')

{'1572959060': '1',
 '1572959175': '1',
 '1572959195': '1',
 '1572959220': '1',
 '1572959225': '1',
 '1572959230': '1'}

In [145]:
conn.zrange('know:',0,-1,withscores=True)

[('18000:hit', 0.0),
 ('1:hit', 0.0),
 ('300:hit', 0.0),
 ('3600:hit', 0.0),
 ('5:hit', 0.0),
 ('60:hit', 0.0),
 ('86400:hit', 0.0)]